In [7]:
import pandas as pd


fn = '/home/hash/code/cest/nature_protocols/OuriCEST_MRF/OptSchedule.xlsx'
df = pd.read_excel(fn)
B1 = df['B1sat'].values
TR = df['TR'].values
Tsat = df['Tsat'].values 
FA = df['FA'].values
off = 3.5 # ppm

f_name = 'scone.txt'

with open(f_name, 'w') as file:
    file.write(f'{len(FA)}\n')

for tr, b1, fa, tsat in zip(TR, B1, FA, Tsat):

    line = f'{tr:.0f} {b1:.2f} {off} {fa} {tsat:.0f}\n'

    with open(f_name, 'a') as file:
        file.write(line)


In [1]:
import numpy as np
import pypulseq as pp
from write_sequence import write_clinical_sequence_gaussian
import os 

f_name = 'scone.txt'

with open(f_name, 'r') as file:
    N = int(file.readline())
    TR, B1, ppm, FA, Tsat = np.zeros((5, N))
    
    for i, line in enumerate(file):
        tr, b1, off, fa, tsat = map(float, line.split())
        TR[i], B1[i], ppm[i], FA[i], Tsat[i] = tr, b1, off, fa, tsat

# to seconds
TR = TR / 1000 
Tsat = Tsat / 1000

gamma = 267.5153  # [rad / uT]
seq_fn = '16msGaussianSCONE.seq'
type_s = 'scanner' # type of sequence can be simulator or scanner

seq_defs = {}

seq_defs["num_meas"] = N  # number of repetition

seq_defs["tp"] = 16e-3  # pulse duration [s]
seq_defs["td"] = 0  # interpulse delay [s]
seq_defs["dcsat"] = (seq_defs["tp"]) / (seq_defs["tp"] + seq_defs["td"])  # duty cycle

seq_defs["n_pulses"] = np.ceil(Tsat / (seq_defs["tp"] + seq_defs["td"])).astype(int)  # number of pulses

seq_defs["tsat"] = Tsat  # saturation time [s]
seq_defs["trec"] = TR - seq_defs["tsat"]  # net recovery time [s]

seq_defs["spoiling"] = True

seqid = os.path.splitext(seq_fn)[1][1:]
seq_defs['seq_id_string'] = seqid  # unique seq id

seq_defs['b1pa'] = B1
seq_defs["b1rms"] = seq_defs["b1pa"]

seq_defs["offsets_ppm"] = ppm  # offset list [ppm]

lims = pp.Opts(
    max_grad=40,
    grad_unit="mT/m",
    max_slew=130,
    slew_unit="T/m/s",
    rf_ringdown_time=30e-6,
    rf_dead_time=100e-6,
    rf_raster_time=1e-6,
    gamma=gamma/2/np.pi*1e6,
)

seq_defs["gamma_hz"] = lims.gamma * 1e-6
seq_defs["freq"] = 127.7292
seq_defs['b0'] = seq_defs['freq'] / seq_defs["gamma_hz"]

write_clinical_sequence_gaussian(seq_defs, seq_fn, lims, type = type_s)

Setting num_meas to 30
Setting tp to 0.016
Setting td to 0
Setting dcsat to 1.0
Setting n_pulses to [140 138  59 185  75 215  77 137 237 187 238 148 157 234 140 237 220 239
 194 204 160  56 197  42 201 193 116 214  45  44]
Setting tsat to [2.235 2.203 0.929 2.955 1.188 3.429 1.225 2.189 3.777 2.978 3.806 2.365
 2.499 3.734 2.237 3.777 3.505 3.814 3.1   3.251 2.555 0.89  3.144 0.658
 3.214 3.082 1.854 3.411 0.71  0.691]
Setting trec to [0.819 0.2   0.911 0.17  1.471 0.17  1.669 1.126 0.17  0.17  0.17  0.17
 0.466 0.17  0.709 0.17  0.44  0.17  0.467 0.17  0.206 1.417 0.277 0.689
 0.78  0.551 2.038 0.17  0.17  2.301]
Setting spoiling to True
Setting seq_id_string to seq
Setting b1pa to [1.6 2.2 2.9 0.7 0.4 1.3 0.3 2.3 0.6 0.2 0.9 0.1 1.1 1.1 1.3 2.6 0.4 2.8
 3.9 0.7 1.8 0.  1.4 0.6 0.  0.7 0.3 1.6 3.6 0. ]
Setting b1rms to [1.6 2.2 2.9 0.7 0.4 1.3 0.3 2.3 0.6 0.2 0.9 0.1 1.1 1.1 1.3 2.6 0.4 2.8
 3.9 0.7 1.8 0.  1.4 0.6 0.  0.7 0.3 1.6 3.6 0. ]
Setting offsets_ppm to [3.5 3.5 3.5 3.5 3.5 3